In [1]:
# Append rolling_ta to sys path.
import sys
import os

# Assuming rolling_ta is in the same level as tests/
module_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))

# Add the path to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from ta.volatility import AverageTrueRange
from rolling_ta.volatility import NumbaTrueRange
from rolling_ta.data import CSVLoader

INFO - Cache [numba=True]


In [3]:
csv_loader = CSVLoader()
data = csv_loader.read_resource()

DEBUG - CSVLoader: Loading from resources/btc_ohlcv.csv


In [4]:
slice_a = data.copy().iloc[:100]
slice_b = data.copy().iloc[100:]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  10000 non-null  float64
 1   open       10000 non-null  float64
 2   high       10000 non-null  float64
 3   low        10000 non-null  float64
 4   close      10000 non-null  float64
 5   volume     10000 non-null  float64
dtypes: float64(6)
memory usage: 468.9 KB


In [6]:
tr = NumbaTrueRange(data)
tr.update(data.iloc[0])

2202.5

In [7]:
%%timeit
AverageTrueRange._true_range(data["high"], data["low"], data["close"].shift(1))

1.92 ms ± 49.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%%timeit
NumbaTrueRange(data, memory=False)

71.4 μs ± 1.26 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
%%timeit
rolling = NumbaTrueRange(slice_a)
for _, series in slice_b.iterrows():
    rolling.update(series)

283 ms ± 3.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
